<a href="https://colab.research.google.com/github/AbdulSheffa/IRP/blob/main/FYP_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
%cd pytorch-CycleGAN-and-pix2pix


Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516 (from 1)
Receiving objects: 100% (2516/2516), 8.20 MiB | 35.14 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
/content/pytorch-CycleGAN-and-pix2pix


In [3]:
!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.5 MB/s eta 0:00:00
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sh

In [46]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image
from PIL import Image
from models.networks import ResnetGenerator, NLayerDiscriminator

# ------------------------------
# 1. Dataset with Self-Supervised Learning & Masking
# ------------------------------
class RainDatasetSSL(Dataset):
    def __init__(self, dataset_name, input_dir, target_dir, transform=None, sample_size=None, output_dir=None, save_masked=False, epoch=0):
        self.dataset_name = dataset_name
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.transform = transform
        self.output_dir = output_dir
        self.save_masked = save_masked
        self.epoch = epoch

        self.input_images = sorted(os.listdir(input_dir))
        self.target_images = sorted(os.listdir(target_dir))

        if sample_size:
            indices = random.sample(range(len(self.input_images)), sample_size)
            self.input_images = [self.input_images[i] for i in indices]
            self.target_images = [self.target_images[i] for i in indices]

        if save_masked and output_dir:
            os.makedirs(output_dir, exist_ok=True)
            self.masked_images_to_save = random.sample(self.input_images, min(10, len(self.input_images)))  # Select 10 random images

    def __len__(self):
        return len(self.input_images)

    def mask_image(self, img_tensor, img_name):
        """Apply dynamic masking for self-supervised learning & save masked image if required."""
        if img_tensor.dim() == 4:  # If batch, process each image separately
            return torch.stack([self.mask_image(single_img, img_name) for single_img in img_tensor])

        if img_tensor.dim() != 3:
            raise ValueError(f"Unexpected tensor shape {img_tensor.shape}, expected (C, H, W)")

        _, h, w = img_tensor.shape
        mask_size = (random.randint(h // 6, h // 4), random.randint(w // 6, w // 4))
        x, y = random.randint(0, h - mask_size[0]), random.randint(0, w - mask_size[1])

        masked_tensor = img_tensor.clone()
        masked_tensor[:, x:x + mask_size[0], y:y + mask_size[1]] = -1

        # Save masked image only if in the selected list
        if self.save_masked and self.output_dir and img_name in self.masked_images_to_save:
            save_path = os.path.join(self.output_dir, f"masked_{self.dataset_name}_epoch_{self.epoch}_{img_name}")
            save_image((masked_tensor + 1) / 2, save_path)  # Denormalizing before saving
            print(f"✅ Saved masked image: {save_path}")

        return masked_tensor

    def __getitem__(self, idx):
        input_img_path = os.path.join(self.input_dir, self.input_images[idx])
        target_img_path = os.path.join(self.target_dir, self.target_images[idx])

        input_image = Image.open(input_img_path).convert("RGB")
        target_image = Image.open(target_img_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        masked_image = self.mask_image(input_image.clone(), self.input_images[idx])
        return masked_image, input_image, target_image, self.input_images[idx]  # ✅ Ensure 4 elements

# ------------------------------
# 2. Masked GAN-Based DeRain CycleGAN
# ------------------------------
class DeRainCycleGANSSL:
    def __init__(self, input_nc, output_nc, ngf, ndf, device):
        self.device = device
        self.netG_A = ResnetGenerator(input_nc, output_nc, ngf).to(device)
        self.netG_B = ResnetGenerator(output_nc, input_nc, ngf).to(device)
        self.netD_A = NLayerDiscriminator(output_nc, ndf).to(device)
        self.netD_B = NLayerDiscriminator(input_nc, ndf).to(device)

        self.criterionGAN = nn.MSELoss()
        self.criterionCycle = nn.L1Loss()
        self.criterionPerceptual = nn.MSELoss()

        self.vgg = models.vgg19(weights=models.VGG19_Weights.DEFAULT).features[:16].eval().to(device)
        for param in self.vgg.parameters():
            param.requires_grad = False

        self.optimizer_G = torch.optim.Adam(
            list(self.netG_A.parameters()) + list(self.netG_B.parameters()), lr=0.00005, betas=(0.5, 0.999)
        )
        self.optimizer_D = torch.optim.Adam(
            list(self.netD_A.parameters()) + list(self.netD_B.parameters()), lr=0.00005, betas=(0.5, 0.999)
        )

    def perceptual_loss(self, x, y):
        return self.criterionPerceptual(self.vgg(x), self.vgg(y))

    def forward(self, real_A, real_B):
        fake_B = self.netG_A(real_A)
        rec_A = self.netG_B(fake_B)
        fake_A = self.netG_B(real_B)
        rec_B = self.netG_A(fake_A)
        return fake_B, rec_A, fake_A, rec_B

    def optimize_parameters(self, real_A, real_B, scaler):
        """Optimizes generator and discriminator parameters for one iteration."""

        self.optimizer_G.zero_grad()

        with torch.amp.autocast(device_type="cuda"):
            fake_B, rec_A, fake_A, rec_B = self.forward(real_A, real_B)
            loss_G_A = self.criterionGAN(self.netD_A(fake_B), torch.ones_like(self.netD_A(fake_B)))
            loss_G_B = self.criterionGAN(self.netD_B(fake_A), torch.ones_like(self.netD_B(fake_A)))
            loss_cycle_A = self.criterionCycle(rec_A, real_A) * 15.0
            loss_cycle_B = self.criterionCycle(rec_B, real_B) * 15.0
            loss_perceptual = self.perceptual_loss(fake_B, real_B) * 0.2
            loss_G = loss_G_A + loss_G_B + loss_cycle_A + loss_cycle_B + loss_perceptual

        # ✅ Check if any loss is NaN or None before backprop
        if any(loss is None or torch.isnan(loss).any() for loss in [loss_G_A, loss_G_B, loss_cycle_A, loss_cycle_B, loss_perceptual]):
            print("❌ Error: One or more loss values are NaN or None!")
            return torch.tensor(0.0, device=self.device), torch.tensor(0.0, device=self.device), torch.tensor(0.0, device=self.device)

        scaler.scale(loss_G).backward()
        scaler.step(self.optimizer_G)
        scaler.update()

        self.optimizer_D.zero_grad()

        with torch.amp.autocast(device_type="cuda"):
            loss_D_A = self.criterionGAN(self.netD_A(real_B), torch.ones_like(self.netD_A(real_B)))
            loss_D_B = self.criterionGAN(self.netD_B(real_A), torch.ones_like(self.netD_B(real_A)))
            loss_D = (loss_D_A + loss_D_B) * 0.5

        # ✅ Check for NaN or None in Discriminator losses
        if any(loss is None or torch.isnan(loss).any() for loss in [loss_D_A, loss_D_B]):
            print("❌ Error: One or more discriminator loss values are NaN or None!")
            return torch.tensor(0.0, device=self.device), torch.tensor(0.0, device=self.device), torch.tensor(0.0, device=self.device)

        scaler.scale(loss_D).backward()
        scaler.step(self.optimizer_D)
        scaler.update()

        # ✅ Convert losses to tensors before returning
        return loss_G.detach(), loss_D_A.detach(), loss_D_B.detach()




In [47]:
import os
import torch
import torch.nn as nn
import torch.cuda.amp as amp  # Mixed Precision Training
from torchvision.transforms.functional import to_pil_image  # For correct image saving
from torchvision import transforms
from torch.utils.data import DataLoader
from models.networks import ResnetGenerator, NLayerDiscriminator

# Define dataset paths and batch sizes
datasets = {
    "Rain100L": {
        "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/input",
        "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/target",
        "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/results_2/",
        "batch_size": 8,  # Custom batch size per dataset
        "sample_size": 100  # Take only 100 samples
    }
    # "DID-MDN-Heavy": {
    #     "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Heavy/rainy",
    #     "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Heavy/non_rainy",
    #     "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_1/",
    #     "batch_size": 6,
    #     "sample_size": 200  # Take 200 samples
    # },
    # "DID-MDN-Medium": {
    # "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Medium/rainy",
    # "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Medium/non_rainy",
    # "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Medium-rain/results_1/",
    # "batch_size": 6,
    # "sample_size": 200  # Take 200 samples
    # },
    # "DID-MDN-Medium": {
    # "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/rainy",
    # "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/non_rainy",
    # "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Light-rain/results_1/",
    # "batch_size": 6,
    # "sample_size": 200  # Take 200 samples
    # }
}

# Image transformations with fixed normalization
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Ensures uniform image size
    transforms.CenterCrop((224, 224)),  # Keeps input-output pairs aligned
    transforms.RandomHorizontalFlip(p=0.5),  # Augmentation
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05),  # Subtle augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25])
])

def denormalize(tensor):
    """Convert tensor from [-1,1] range to [0,1] for correct visualization"""
    return torch.clamp(tensor * 0.5 + 0.5, 0, 1)

# Initialize Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize Model
model = DeRainCycleGANSSL(input_nc=3, output_nc=3, ngf=64, ndf=64, device=device)

# Learning Rate Schedulers
scheduler_G = torch.optim.lr_scheduler.StepLR(model.optimizer_G, step_size=20, gamma=0.5)
scheduler_D = torch.optim.lr_scheduler.StepLR(model.optimizer_D, step_size=20, gamma=0.5)

# Training Configuration
num_epochs = 70
checkpoint_dir = "/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)  # Ensure checkpoint folder exists

# Mixed Precision Training
scaler = amp.GradScaler()

# Loop Through Each Dataset and Train
for dataset_name, paths in datasets.items():
    print(f"\n🔹 Training on {dataset_name} dataset...")

    input_dir = paths["input"]
    target_dir = paths["target"]
    output_dir = paths["output"]
    batch_size = paths["batch_size"]
    sample_size = paths["sample_size"]

    os.makedirs(output_dir, exist_ok=True)

    # Load dataset with masked saving enabled
    train_loader = DataLoader(
        RainDatasetSSL(
            dataset_name=dataset_name,
            input_dir=input_dir,
            target_dir=target_dir,
            transform=transform,
            sample_size=sample_size,
            output_dir=output_dir,
            save_masked=True
        ),
        batch_size=batch_size,
        shuffle=True
    )

    for epoch in range(num_epochs):
        for i, (masked_A, real_A, real_B, img_name) in enumerate(train_loader):
            masked_A, real_A, real_B = masked_A.to(device), real_A.to(device), real_B.to(device)

            # Optimize model and check loss
            loss_G, loss_D_A, loss_D_B = model.optimize_parameters(masked_A, real_B, scaler)

            # ✅ Ensure all loss values are valid tensors
            if not isinstance(loss_G, torch.Tensor) or not isinstance(loss_D_A, torch.Tensor) or not isinstance(loss_D_B, torch.Tensor):
                print(f"⚠️ Error: Loss values must be tensors! Received: loss_G={loss_G}, loss_D_A={loss_D_A}, loss_D_B={loss_D_B}")
                continue

            # ✅ Check for NaN values in loss
            if torch.isnan(loss_G).any() or torch.isnan(loss_D_A).any() or torch.isnan(loss_D_B).any():
                print("⚠️ NaN detected in loss! Skipping update")
                continue

            print(f"Epoch {epoch}/{num_epochs}, Batch {i}/{len(train_loader)}, Image: {img_name[0]}, "
                  f"Loss_G: {loss_G.item():.4f}, Loss_D_A: {loss_D_A.item():.4f}, Loss_D_B: {loss_D_B.item():.4f}")

        scheduler_G.step()
        scheduler_D.step()
        torch.cuda.empty_cache()  # Prevent OOM errors

        # ✅ Save checkpoint every 5 epochs
        if epoch % 5 == 0 or epoch == num_epochs - 1:
            torch.save({
                'epoch': epoch,
                'model_G_A_state_dict': model.netG_A.state_dict(),
                'model_G_B_state_dict': model.netG_B.state_dict(),
                'model_D_A_state_dict': model.netD_A.state_dict(),
                'model_D_B_state_dict': model.netD_B.state_dict(),
                'optimizer_G_state_dict': model.optimizer_G.state_dict(),
                'optimizer_D_state_dict': model.optimizer_D.state_dict(),
                'scheduler_G_state_dict': scheduler_G.state_dict(),
                'scheduler_D_state_dict': scheduler_D.state_dict(),
                'scaler_state_dict': scaler.state_dict()
            }, os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth"))
            print(f"✅ Checkpoint saved at epoch {epoch}")

    # ✅ Save final generated images with correct colors
    if epoch == num_epochs - 1:
        model.netG_A.eval()
        with torch.no_grad():
            for i, (_, real_A, _, img_names) in enumerate(train_loader):
                real_A = real_A.to(device)
                fake_B = model.netG_A(real_A)
                fake_B = denormalize(fake_B)

                for j in range(real_A.size(0)):
                    img_name = img_names[j]
                    save_path = os.path.join(output_dir, f"{dataset_name}_epoch_{epoch}_de_rained_{img_name}")

                    # ✅ Convert to PIL and save
                    pil_img = to_pil_image(fake_B[j].cpu())  # Ensures correct color space
                    pil_img.save(save_path)

                    print(f"✅ Saved: {save_path}")

print("Training on all datasets completed ✅")


Using device: cuda


<ipython-input-47-c8a3cadff606>:52: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()



🔹 Training on Rain100L dataset...
✅ Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/results_6/masked_Rain100L_epoch_0_56.png
Epoch 0/70, Batch 0/13, Image: 25.png, Loss_G: 30.3775, Loss_D_A: 0.9213, Loss_D_B: 1.3365
Epoch 0/70, Batch 1/13, Image: 35.png, Loss_G: 28.5854, Loss_D_A: 0.7289, Loss_D_B: 1.3643
✅ Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/results_6/masked_Rain100L_epoch_0_15.png
✅ Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/results_6/masked_Rain100L_epoch_0_1.png
Epoch 0/70, Batch 2/13, Image: 53.png, Loss_G: 25.4160, Loss_D_A: 0.7825, Loss_D_B: 0.7511
Epoch 0/70, Batch 3/13, Image: 96.png, Loss_G: 29.8765, Loss_D_A: 0.2980, Loss_D_B: 0.6789
Epoch 0/70, Batch 4/13, Image: 66.png, Loss_G: 27.2156, Loss_D_A: 0.3168, Loss_D_B: 0.3943
Epoch 0/70, Batch 5/13, Image: 75.png, Loss_G: 24.9783, Loss_D_A: 0.1772, Loss_D_B: 0.2917
✅ Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/Ra

In [48]:
import os
import torch
import torch.nn as nn
from torchvision.utils import save_image
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# ------------------------------
# 🔹 Define Autoencoder with Skip Connections
# ------------------------------
class AutoencoderWithSkipConnections(nn.Module):
    def __init__(self, input_nc):
        super(AutoencoderWithSkipConnections, self).__init__()
        self.encoder1 = nn.Sequential(nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.encoder2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.encoder3 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.bottleneck = nn.Sequential(nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder1 = nn.Sequential(nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder2 = nn.Sequential(nn.ConvTranspose2d(256 + 256, 128, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder3 = nn.Sequential(nn.ConvTranspose2d(128 + 128, 64, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder4 = nn.Sequential(nn.ConvTranspose2d(64 + 64, input_nc, kernel_size=4, stride=2, padding=1), nn.Tanh())

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        b = self.bottleneck(e3)
        d1 = self.decoder1(b)
        d2 = self.decoder2(torch.cat([d1, e3], dim=1))
        d3 = self.decoder3(torch.cat([d2, e2], dim=1))
        d4 = self.decoder4(torch.cat([d3, e1], dim=1))
        return d4

# ------------------------------
# 🔹 Perceptual Loss using VGG19
# ------------------------------
class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        vgg19 = models.vgg19(pretrained=True).features[:16].eval()
        for param in vgg19.parameters():
            param.requires_grad = False
        self.vgg19 = vgg19
        self.criterion = nn.MSELoss()

    def forward(self, x, y):
        return self.criterion(self.vgg19(x), self.vgg19(y))

# ------------------------------
# 🔹 Custom Dataset to Maintain Filenames
# ------------------------------
class ImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_filenames = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, self.image_filenames[idx]  # ✅ Return image + filename

# ------------------------------
# 🔹 Initialize Autoencoder and Loss Functions
# ------------------------------
input_channels = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = AutoencoderWithSkipConnections(input_channels).to(device)

mse_loss = nn.MSELoss()
perceptual_loss = PerceptualLoss().to(device)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0002)

# ------------------------------
# 🔹 Define GAN-generated output directories
# ------------------------------
gan_output_dirs = {
    "results_6": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/results_6",
}

# Image Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# ------------------------------
# 🔹 Process each GAN-generated directory
# ------------------------------
for dir_name, gan_output_path in gan_output_dirs.items():
    print(f"\n🔹 Processing GAN outputs from: {gan_output_path}")

    # ✅ Use Custom Dataset to Track Filenames
    dataset = ImageDataset(gan_output_path, transform=transform)
    train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

    epochs = 60
    reconstructed_images = {}

    # ------------------------------
    # 🔹 Train Autoencoder
    # ------------------------------
    for epoch in range(epochs):
        for images, filenames in train_loader:  # ✅ Filenames are now correctly mapped
            images = images.to(device)
            reconstructed = autoencoder(images)

            mse_loss_value = mse_loss(reconstructed, images)
            perceptual_loss_value = perceptual_loss(reconstructed, images)
            total_loss = mse_loss_value + 0.01 * perceptual_loss_value

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            if epoch == epochs - 1:
                for j, filename in enumerate(filenames):
                    reconstructed_images[filename] = reconstructed[j].unsqueeze(0)

        print(f"Epoch [{epoch+1}/{epochs}], MSE Loss: {mse_loss_value.item():.4f}, Perceptual Loss: {perceptual_loss_value.item():.4f}")

    # ------------------------------
    # 🔹 Save Reconstructed Images
    # ------------------------------
    output_dir = f"/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/reconstructed_autoencoder_{dir_name}"
    os.makedirs(output_dir, exist_ok=True)

    for filename, reconstructed_image in reconstructed_images.items():
        save_path = os.path.join(output_dir, f"{filename.split('.')[0]}_refined.png")
        save_image(reconstructed_image, save_path)

    print(f"✅ Finished processing {dir_name} - Saved to: {output_dir}")

print("\n🎯 Autoencoder Training and Refinement Completed for All GAN Output Directories!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



🔹 Processing GAN outputs from: /content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/results_6
Epoch [1/60], MSE Loss: 0.1456, Perceptual Loss: 5.1418
Epoch [2/60], MSE Loss: 0.0560, Perceptual Loss: 3.4991
Epoch [3/60], MSE Loss: 0.0384, Perceptual Loss: 3.4001
Epoch [4/60], MSE Loss: 0.0196, Perceptual Loss: 2.4026
Epoch [5/60], MSE Loss: 0.0207, Perceptual Loss: 2.2471
Epoch [6/60], MSE Loss: 0.0253, Perceptual Loss: 3.1974
Epoch [7/60], MSE Loss: 0.0134, Perceptual Loss: 1.6662
Epoch [8/60], MSE Loss: 0.0172, Perceptual Loss: 2.1239
Epoch [9/60], MSE Loss: 0.0110, Perceptual Loss: 1.8031
Epoch [10/60], MSE Loss: 0.0119, Perceptual Loss: 1.5214
Epoch [11/60], MSE Loss: 0.0141, Perceptual Loss: 1.9234
Epoch [12/60], MSE Loss: 0.0064, Perceptual Loss: 0.9545
Epoch [13/60], MSE Loss: 0.0074, Perceptual Loss: 1.1230
Epoch [14/60], MSE Loss: 0.0050, Perceptual Loss: 0.8228
Epoch [15/60], MSE Loss: 0.0043, Perceptual Loss: 0.6699
Epoch [16/60], MSE Loss: 0.0056, Perceptual Loss: 1.0720
E

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# If a GPU is available, you can also check its details
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"GPU Memory Cached: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")


Using device: cuda
GPU Name: Tesla T4
GPU Memory Allocated: 0.00 GB
GPU Memory Cached: 0.00 GB
